In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
import torchvision.utils as vutils
from torch.autograd import Variable
from tqdm.notebook import tqdm_notebook
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms
import random
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
from PIL import Image
import cv2
import time
from tqdm.notebook import tqdm_notebook
import time
import os, shutil
import matplotlib.animation as animation
from mpl_toolkits.axes_grid1 import ImageGrid
import sys, time
from nibabel.testing import data_path
import nibabel as nib
from PIL import Image
import scipy.ndimage as ndi
import itertools
import time
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
import tqdm
import nltk
nltk.download('punkt')
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ragha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
device = torch.device("cuda")

True
1


In [5]:
data = pd.read_csv(r"C:\Users\ragha\Documents\CSCI 544\project\FinalDataset.csv")
data.head()

,sentence,class
0,What city in the United States has the highest...,0
1,"At work, wishing I was out on the boat",0
2,A smile is a curve that sets everything straig...,0
3,Does sleep quality mediate the association bet...,0
4,What city was found on the west bank of the ri...,0


In [6]:
data['class'].value_counts()

0    400015
1    376930
Name: class, dtype: int64

In [7]:
train_data, test_data = train_test_split(data, test_size=0.1, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.1, random_state=42)

In [8]:
train_data

,sentence,class
121746,@VPG_Printing ok promise....I will email in th...,0
442447,Can the absence of the glycine N-methyltransfe...,1
706803,"brown"" race=""nonwhite"" position=""DB"">sergio br...",1
107317,Who must be presented with every bill passed b...,0
121324,@jessieshmessie You should bring them over! @w...,0
...,...,...
129254,my face feels amazing right now... did a salt ...,0
25876,Are transposable elements enriched within or i...,0
522322,What's the reason behind the GLAM SQUAD's spec...,1
21294,"Shaq is gonna fail on the Cavs, and my Reds lo...",0


In [9]:
print('Train:', len(train_data))
print('Val:', len(val_data))
print('Test:', len(test_data))

Train: 629325
Val: 69925
Test: 77695


In [10]:
train_data['class'].value_counts()

0    324086
1    305239
Name: class, dtype: int64

In [11]:
val_data['class'].value_counts()

0    36099
1    33826
Name: class, dtype: int64

In [12]:
test_data['class'].value_counts()

0    39830
1    37865
Name: class, dtype: int64

In [13]:
# CREATE DATASET CLASS FOR DATALOADERS
class Dataset(Dataset):
    def __init__(self, dataframe):
        self.texts = dataframe['sentence'].values.tolist()
        self.labels = dataframe['class'].values.tolist()
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        X = self.texts[idx]
        y = self.labels[idx]
        X = X.lower()
        tokens = nltk.word_tokenize(X, language="english")
        X = " ".join(tokens)
        X = X.strip()
        
        return X, y

In [14]:
class transformer(nn.Module):
    def __init__(self, base_model):
        super(transformer, self).__init__()

        self.bert = base_model
        self.fc1 = nn.Linear(768, 32)
        self.fc2 = nn.Linear(32, 1)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, input_ids, attention_mask):
        bert_out = self.bert(input_ids=input_ids,
                             attention_mask=attention_mask)[0][:, 0]
        x = self.fc1(bert_out)
        x = self.relu(x)
        
        x = self.fc2(x)
        x = self.sigmoid(x)

        return x

In [15]:
torch.manual_seed(0)

BERT_MODEL = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL)
base_model = AutoModel.from_pretrained(BERT_MODEL)

model = transformer(base_model)
model = model.to(device)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
def collate_fn(batch):    
    x, y = [], []
    for text, label in batch:
        x.append(text)
        y.append(label)
    padded_text = tokenizer(x, padding=True, truncation=True, return_tensors='pt')
    
    return padded_text, torch.FloatTensor(y)

In [15]:
train_dataloader = DataLoader(Dataset(train_data), batch_size=20, shuffle=True, num_workers=2, collate_fn=collate_fn, pin_memory=True)
val_dataloader = DataLoader(Dataset(val_data), batch_size=20, num_workers=2, collate_fn=collate_fn, pin_memory=True)

In [16]:
for train_input, train_label in train_dataloader:
    print(np.array(train_input['input_ids']).shape)
    break

(20, 269)


In [17]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

In [18]:
best_val_loss = float('inf')
early_stopping_threshold_count = 0

for epoch in range(5):
    total_acc_train = 0
    total_loss_train = 0

    model.train()

    for train_input, train_label in train_dataloader:
        
        attention_mask = train_input['attention_mask'].to(device)
        input_ids = train_input['input_ids'].squeeze(1).to(device)
        train_label = train_label.to(device)

        output = model(input_ids, attention_mask)

        loss = criterion(output, train_label.float().unsqueeze(1))

        total_loss_train += loss.item()
        
        pred = (output >= 0.5).int()
        label = train_label.unsqueeze(1)
        auc_roc = roc_auc_score(label.cpu(), pred.cpu())
        total_acc_train += auc_roc

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    with torch.no_grad():
        total_acc_val = 0
        total_loss_val = 0

        model.eval()

        for val_input, val_label in val_dataloader:
            attention_mask = val_input['attention_mask'].to(device)
            input_ids = val_input['input_ids'].squeeze(1).to(device)

            val_label = val_label.to(device)

            output = model(input_ids, attention_mask)

            loss = criterion(output, val_label.float().unsqueeze(1))

            total_loss_val += loss.item()

            pred = (output >= 0.5).int()
            label = val_label.unsqueeze(1)
            auc_roc = roc_auc_score(label.cpu(), pred.cpu())
            total_acc_val += auc_roc

        print("Epochs:", epoch + 1)
        print("Train Loss:", total_loss_train/len(train_dataloader))
        print("Train AUCROC:", total_acc_train/len(train_dataloader))
        print("Val Loss:", total_loss_val/len(val_dataloader))
        print("Val AUCROC:", total_acc_val/len(val_dataloader))
        print("====================================================================================")

        if best_val_loss > total_loss_val:
            best_val_loss = total_loss_val
            torch.save(model, f"best_model.pt")
            print("Saved model")
            early_stopping_threshold_count = 0
        else:
            early_stopping_threshold_count += 1

        if early_stopping_threshold_count >= 1:
            print("Early stopping")
            break

Epochs: 1
Train Loss: 0.15413501266408938
Train AUCROC: 0.9366514146432944
Val Loss: 0.12856202599141295
Val AUCROC: 0.9504340627095027
Saved model
Epochs: 2
Train Loss: 0.10370486801399903
Train AUCROC: 0.9588303768172022
Val Loss: 0.10831817551712075
Val AUCROC: 0.9572462066945798
Saved model
Epochs: 3
Train Loss: 0.0803500216851612
Train AUCROC: 0.9684961394591337
Val Loss: 0.13090185478835434
Val AUCROC: 0.9538917120010817
Early stopping
